Connected to PRL (Python 3.11.0)

In [2]:
# Define the intervals for gamma and reward shaping factor
gamma_interval = [0.93, 0.99]
reward_shaping_interval = [0.5, 0.9]
peakvalley = [True, False]
# #battery_factor_interval = [0.0, 0.15]

# # Define the number of iterations for the random search
num_iterations = 100

for i in range(num_iterations):
    
    # Generate random values within the intervals
    gamma = round(np.random.uniform(*gamma_interval), 4)
    factor = round(np.random.uniform(*reward_shaping_interval), 4)
    pv = random.choice(peakvalley)
    battery_factor = None #round(np.random.uniform(*battery_factor_interval), 4)
    
    print(f'\nIteration: {i+1}, Gamma: {gamma}, Reward Shaping Factor: {factor}, Peak and Valley: {pv}')
        
    seed = 2705
    rep = 105000 * 3
    batch_size = 48
    gamma = gamma
    epsilon = 1.0
    epsilon_decay = 99999
    epsilon_min = 0.1
    learning_rate = 5e-5
    price_horizon = 48
    future_horizon = 0
    hidden_dim = 128
    num_layers = 4
    positions = False
    action_classes = 3
    reward_shaping = True
    implicit_shape = True
    factor = factor
    verbose = False
    normalize = True
    df = train_name


    # Initialize Environment
    env = Electric_Car(path_to_test_data=df)
    val_env = Electric_Car(path_to_test_data=val_name)

    #Initialize DQN
    agent = DDQNAgent(env = env,
                    features = features_train,
                    epsilon_decay = epsilon_decay,
                    epsilon_start = epsilon,
                    epsilon_end = epsilon_min,
                    discount_rate = gamma,
                    lr = learning_rate,
                    buffer_size = 100000,
                    price_horizon = price_horizon,
                    hidden_dim=hidden_dim,
                    num_layers = num_layers,
                    positions = positions,
                    action_classes = action_classes, 
                    reward_shaping = reward_shaping,
                    implicit_shape = implicit_shape,
                    shaping_factor = factor,
                    normalize = normalize,
                    verbose = verbose)

    val_agent = DDQNAgent(env = val_env,
                    features = features_val,
                    epsilon_decay = epsilon_decay,
                    epsilon_start = epsilon,
                    epsilon_end = epsilon_min,
                    discount_rate = gamma,
                    lr = learning_rate,
                    buffer_size = 100000,
                    price_horizon = price_horizon,
                    hidden_dim=hidden_dim,
                    num_layers = num_layers,
                    positions = positions,
                    action_classes = action_classes, 
                    reward_shaping = reward_shaping,
                    implicit_shape = implicit_shape,
                    shaping_factor = factor,
                    normalize = normalize,
                    verbose = verbose)


    episode_balance = 0
    episode_loss = 0
    episode_counter = 0
    episode_reward = 0

    obs,_,_,_,_ = env.step(0) # First do nothing
    state = agent.obs_to_state(obs)
    
    for i in tqdm(range(rep)):

        action, q = agent.choose_action(i, state, greedy = False) # Choose action (discrete)
        cont_action = agent.action_to_cont(action) # Convert to continuous action
        
        # Get new observation and state
        new_obs, r, t, _, _ = env.step(cont_action)
        new_state = agent.obs_to_state(new_obs)
        
        # Reward Shaping            
        new_reward = agent.shape_reward(r, cont_action, peakvalley = pv)

        # Fill replay buffer - THIS IS THE ONLY THING WE DO WITH THE CURRENT OBSERVATION - LEARNING IS FULLY PERFORMED FROM THE REPLAY BUFFER
        if state.shape[0] == agent.state_dim and new_state.shape[0] == agent.state_dim:
            agent.replay_memory.add_data((state, action, new_reward, t, new_state))

        #Update DQN
        loss = agent.optimize(batch_size)
        
        # Update values
        episode_balance += r
        episode_reward += r
        episode_loss += loss

        # New observation
        state = new_state        

        if t:
            # Reset Environment
            env.counter = 0
            env.hour = 1
            env.day = 1
            episode_counter += 1
            
            if episode_counter % 4 == 0:
                print('Episode ', episode_counter, 'Balance: ', episode_balance, 'Reward: ', episode_reward, 'Loss: ', episode_loss) # Add both balance and reward to see how training objective and actually spent money differ
            
            # Scheduler Step
            agent.scheduler.step(episode_loss)
            
            episode_loss = 0
            episode_balance = 0
            episode_reward = 0
            
            
            if episode_counter % 4 == 0:
                # Evaluate DQN - Training
                train_dqn = DDQNEvaluation(price_horizon = price_horizon)
                _ = train_dqn.evaluate(agent = agent)
                
                # Evaluate DQN - Validation
                val_agent.dqn_predict.load_state_dict(agent.dqn_predict.state_dict())
                val_dqn = DDQNEvaluation(price_horizon = price_horizon)
                balance = val_dqn.evaluate(agent = val_agent, validation = True)
                
                # Reset Environments after evaluation
                env.counter = 0
                env.hour = 1
                env.day = 1
                val_env.counter = 0
                val_env.hour = 1
                val_env.day = 1
                
                if balance < -650:
                    print("Balance too low, stopping training")
                    #break
            
                
    # Save agent
    torch.save(agent.dqn_predict.state_dict(), f'models/change_tuning_gamma_{gamma}_valley_{pv}.pt')


Iteration: 1, Gamma: 0.949, Reward Shaping Factor: 0.5818, Peak and Valley: True
Number of engineered features:  57
State dimension:  113
Number of DQN Parameters:  48003
Number of engineered features:  57
State dimension:  113
Number of DQN Parameters:  48003


 33%|███▎      | 104153/315000 [06:13<21:42, 161.84it/s]

Episode  4 Balance:  -2180.5101852469056 Reward:  -2180.5101852469056 Loss:  1177.8178064418025
Training Balance:  -443.1112687037038


 33%|███▎      | 104188/315000 [06:36<15:45:17,  3.72it/s]

Validation Balance:  -783.6901994444445
Balance too low, stopping training


...

Validation Balance:  -621.8566100617285


100%|██████████| 315000/315000 [16:28<00:00, 318.53it/s]



Iteration: 27, Gamma: 0.9693, Reward Shaping Factor: 0.7334, Peak and Valley: True
Number of engineered features:  57
State dimension:  113
Number of DQN Parameters:  48003
Number of engineered features:  57
State dimension:  113
Number of DQN Parameters:  48003


 26%|██▋       | 82791/315000 [05:22<15:03, 256.89it/s]


KeyboardInterrupt: 